<center><span style="font-size:20px;font-weight:bold;color:black;">第三次作业</span></center>      
$\quad$
<center>朱强强</center>
<center>17064001</center>
<center>应用统计学1701</center>

第二次作业是用来kNN决策树的方法对乳腺癌数据做了分类预测，本次作业使用的是Logistic回归算法进行预测。         
数据链接：https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

### 数据预处理
数据预处理步骤与第二次作业的处理相似。

In [13]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv("data.csv")
print("数据的维度是: ", data.shape)
data.head(10)

数据的维度是:  (569, 33)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,NaN
7,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,NaN
8,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,NaN
9,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,NaN


从上面数据我们可以看出，id对我们的分析并无实质作用，diagnosis是我们的目标变量，最后一列的unnamed:32全是空值。我们将diagnosis这一列提取出来，然后将这三列删除。

In [14]:
y = data.diagnosis
feature_elimination_list = ['id', 'diagnosis', 'Unnamed: 32']
X = data.drop(feature_elimination_list, axis=1)

**对数据进行0-1标准化处理**

In [15]:
def min_max_normalize(x):
    return (x - x.min())/(x.max() - x.min())

In [16]:
for col in x.columns[0:31]:
    x[col] = min_max_normalize(x[col])

In [35]:
# 特征工程
drop_list1 = ['perimeter_mean', 'area_mean', 'compactness_mean', 'concave points_mean', 'radius_se', 'perimeter_se', 'radius_worst',
              'perimeter_worst', 'compactness_worst', 'concave points_worst', 'compactness_se', 'concave points_se', 'texture_worst', 
              'area_worst']
X_new = X.drop(drop_list1, axis=1 ) 
X_new.head()

,radius_mean,texture_mean,smoothness_mean,concavity_mean,symmetry_mean,fractal_dimension_mean,texture_se,area_se,smoothness_se,concavity_se,symmetry_se,fractal_dimension_se,smoothness_worst,concavity_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,0.11840,0.3001,0.2419,0.07871,0.9053,153.40,0.006399,0.05373,0.03003,0.006193,0.1622,0.7119,0.4601,0.11890
1,20.57,17.77,0.08474,0.0869,0.1812,0.05667,0.7339,74.08,0.005225,0.01860,0.01389,0.003532,0.1238,0.2416,0.2750,0.08902
2,19.69,21.25,0.10960,0.1974,0.2069,0.05999,0.7869,94.03,0.006150,0.03832,0.02250,0.004571,0.1444,0.4504,0.3613,0.08758
3,11.42,20.38,0.14250,0.2414,0.2597,0.09744,1.1560,27.23,0.009110,0.05661,0.05963,0.009208,0.2098,0.6869,0.6638,0.17300
4,20.29,14.34,0.10030,0.1980,0.1809,0.05883,0.7813,94.44,0.011490,0.05688,0.01756,0.005115,0.1374,0.4000,0.2364,0.07678


### 建立模型

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

# 查看训练和测试样本的数量和类别分布
print(y_train.value_counts() / len(y_train))
print(y_test.value_counts() / len(y_test))

B    0.625628
M    0.374372
Name: diagnosis, dtype: float64
B    0.631579
M    0.368421
Name: diagnosis, dtype: float64


从上面结果我们可以看出，训练集和测试集中Benign和Malignant的比例大致相等，可以进行模型训练。

In [47]:
from sklearn.linear_model import LogisticRegression

# 惩罚项使用的是L1范数
lr_model = LogisticRegression(penalty='l1', tol=0.01, solver='liblinear')

lr_model.fit(X_train,y_train)
lr_model_predict = lr_model.predict(X_test)

# 性能分析
from sklearn import metrics

print('精确率为：',lr_model.score(X_test,y_test))
print(metrics.classification_report(y_test, lr_model_predict, target_names = ['Benign','Maligant']))

confusion_mat = metrics.confusion_matrix(y_test, lr_model_predict)
print ("假阴性：",confusion_mat[0,1])
print ("假阳性：",confusion_mat[1,0])

精确率为： 0.9649122807017544
              precision    recall  f1-score   support

      Benign       0.98      0.96      0.97       108
    Maligant       0.94      0.97      0.95        63

    accuracy                           0.96       171
   macro avg       0.96      0.97      0.96       171
weighted avg       0.97      0.96      0.97       171

假阴性： 4
假阳性： 2


In [48]:
# 惩罚项使用的是L2范数
lr_model_new = LogisticRegression(penalty='l2', tol=0.01, solver='liblinear')

lr_model_new.fit(X_train,y_train)
lr_model_new_predict = lr_model.predict(X_test)

# 性能分析
print('精确率为：',lr_model_new.score(X_test, y_test))
print(metrics.classification_report(y_test, lr_model_new_predict, target_names = ['Benign','Maligant']))

confusion_mat = metrics.confusion_matrix(y_test, lr_model_new_predict)
print ("假阴性：",confusion_mat[0,1])
print ("假阳性：",confusion_mat[1,0])

精确率为： 0.9415204678362573
              precision    recall  f1-score   support

      Benign       0.98      0.96      0.97       108
    Maligant       0.94      0.97      0.95        63

    accuracy                           0.96       171
   macro avg       0.96      0.97      0.96       171
weighted avg       0.97      0.96      0.97       171

假阴性： 4
假阳性： 2


从模型的精确率来说，惩罚项使用L1范数的精确程度更高。

### 模型优化

In [49]:
from sklearn.preprocessing import StandardScaler

# 标准化数据，保证每个维度的特征数据方差为1，均值为0。使得预测结果不会被某些维度过大的特征值而主导
ss = StandardScaler()
 
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# 惩罚项使用的是L1范数
lr_model = LogisticRegression(penalty='l1', tol=0.01, solver='liblinear')

lr_model.fit(X_train,y_train)
lr_model_predict = lr_model.predict(X_test)

# 性能分析
from sklearn.metrics import classification_report

print('精确率为：',lr_model.score(X_test,y_test))
print(classification_report(y_test, lr_model_predict, target_names = ['Benign','Maligant']))

confusion_mat = metrics.confusion_matrix(y_test, lr_model_predict)
print ("假阴性：",confusion_mat[0,1])
print ("假阳性：",confusion_mat[1,0])

精确率为： 0.9707602339181286
              precision    recall  f1-score   support

      Benign       0.99      0.96      0.98       108
    Maligant       0.94      0.98      0.96        63

    accuracy                           0.97       171
   macro avg       0.96      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171

假阴性： 4
假阳性： 1


In [50]:
# 惩罚项使用的是L2范数
lr_model_new = LogisticRegression(penalty='l2', tol=0.01, solver='liblinear')

lr_model_new.fit(X_train,y_train)
lr_model_new_predict = lr_model.predict(X_test)

# 性能分析
print('精确率为：',lr_model_new.score(X_test, y_test))
print(classification_report(y_test, lr_model_new_predict, target_names = ['Benign','Malignant']))

confusion_mat = metrics.confusion_matrix(y_test, lr_model_new_predict)
print ("假阴性：",confusion_mat[0,1])
print ("假阳性：",confusion_mat[1,0])

精确率为： 0.9766081871345029
              precision    recall  f1-score   support

      Benign       0.99      0.96      0.98       108
   Malignant       0.94      0.98      0.96        63

    accuracy                           0.97       171
   macro avg       0.96      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171

假阴性： 4
假阳性： 1


数据标准化处理的结果比归一化处理的结果要好，不管惩罚项使用的是L1范数还是L2范数均更加精确。  
但数据经过标准化处理之后，模型惩罚项使用L2范数的精确度比使用L1范数的精确度更高，精确率达到了97.66%，且比在第二章作业中使用kNN方法得到的最优的模型的95.91%精确度要高。
总体来说，该数据经过标准化处理且模型惩罚项使用L2范数的模型结果是比较好的。